In [4]:
import math
def deg2rad(deg):
    return deg * math.pi / 180

In [5]:
a = 6378137.0
b = 6356752.3


R = a

In [6]:
def toMerc(lng, lat):
    x = R * deg2rad(lng)
    y = R * math.log(math.tan(math.pi/4 + deg2rad(lat)/2))

    return x,y

In [7]:
toMerc(-180, 0)

(-20037508.342789244, -7.081154551613622e-10)

In [8]:
toMerc(180, 0)

(20037508.342789244, -7.081154551613622e-10)

In [9]:
q = toMerc(180, 0)[0] - toMerc(-180, 0)[0] # Erdumfang

In [10]:
q / (2*math.pi) # ergibt wieder den Erdradius

6378137.000000001

In [11]:
from pyproj import Transformer

wgs84merc = Transformer.from_crs(4326, 3857, always_xy=True)

In [12]:
wgs84merc.transform(-180, 0)

(-20037508.342789244, 0.0)

In [13]:
wgs84merc.transform(180, 0)

(20037508.342789244, 0.0)

In [14]:
mercwgs84 = Transformer.from_crs(3857, 4326, always_xy=True)

In [15]:
mercwgs84.transform(-20037508.342789244, 0.0)

(-180.0, 0.0)

In [16]:
mercwgs84.transform(-20037508.342789244, -20037508.342789244)

(-180.0, -85.0511287798066)

In [17]:
mercwgs84.transform(20037508.342789244, 20037508.342789244)

(180.0, 85.0511287798066)

In [18]:
def QuadKeyToNormalizedMercatorCoord(key):
    zoomlevels = len(key)
    x = 0
    y = 0
    scale = 1.0
    for i in range(0,zoomlevels):
        scale /= 2.0
        if key[i] == "0":
            y += scale
        elif key[i] == "1":
            x += scale
            y += scale
        elif key[i] == "2":
            pass
        elif key[i] == "3":
            x += scale

    return [ 2*x-1, 2*y-1 , 2*(x + scale)-1, 2*(y + scale)-1]


QuadKeyToNormalizedMercatorCoord("2")

# Der Grund warum man [-1, 1] und nicht [-20mio, 20mio] nimmt, ist die Präzision des DT Floatingpoint.
# Je grösser die Zahl ist, umso mehr Präzision verliert man. Die beste Präzision hat man im Bereich [-1, 1].
# Dies betrifft nicht nur Python sondern alle Programmiersprachen. Es ist die Schuld der Definition des Floatingpoint-Standards.

[-1, -1, 0.0, 0.0]

# Wieso [-1, 1] und nicht [-20mio, 20mio]?
Der Grund warum man [-1, 1] und nicht [-20mio, 20mio] nimmt, ist die Präzision des DT Floatingpoint. <br/>
Je grösser die Zahl ist, umso mehr Präzision verliert man. Die beste Präzision hat man im Bereich [-1, 1].<br/>
Dies betrifft nicht nur Python sondern alle Programmiersprachen. Es ist die Schuld der Definition des Floatingpoint-Standards.

In [19]:
QuadKeyToNormalizedMercatorCoord("0")

[-1, 0.0, 0.0, 1.0]

# Umrechnung Tilekoordinaten nach Quadkey

In [20]:
def Tilecoord2Quadkey(x, y, zoom):
    d = 0
    quadkey = ""
    for i in range(zoom, 0, -1):
        mask = 1 << (i-1) # << ist das Bitshifting
        if (x & mask) != 0:
            d += 1
        if (y & mask) != 0:
            d += 2
        quadkey += str(d)
    return quadkey

In [21]:
Tilecoord2Quadkey(1,3,5)

'00025'

In [22]:
# EPSG:3857 -> EPSG:4326
def MercatorToWGS84(x, y):
    a = 6_378_137
    x = x/a
    y = y/a

    t = math.exp(-y)
    lat = math.pi/2 - 2 * math.atan(t)
    lng = x

    return [lng * 180 / math.pi, lat * 180 / math.pi]

In [23]:
MercatorToWGS84(20_000_000, 0)

[179.6630568239043, 0.0]

In [24]:
def NormalizedMercatorToWGS84(x, y):
    x = x * math.pi
    y = y * math.pi

    t = math.exp(-y)
    lat = math.pi/2 - 2 * math.atan(t)
    lng = x

    return [lng * 180 / math.pi, lat * 180 / math.pi]

In [25]:
NormalizedMercatorToWGS84(-1, -1)

[-180.0, -85.05112877980659]

In [26]:
def QuadkeyToWGS84(quadcode):
    merc = QuadKeyToNormalizedMercatorCoord(quadcode)
    t0 = NormalizedMercatorToWGS84(merc[0], merc[2])
    t1 = NormalizedMercatorToWGS84(merc[1], merc[3])
    return t0, t1


In [27]:
QuadkeyToWGS84("1012301203012301023010230120301230201")

([59.53030495700659, 51.031588324664746],
 [144.42414173012367, 80.80602508251606])

In [28]:
def WGS84ToTile(lng, lat, zoom):
    mapsize = 2**zoom # returns with = heigt

    x = int((lng + 180) / 360 * 2**zoom) # int weil man ganze Zahlen will für die Tiles
    y = int((1 - math.log(math.tan(lat * math.pi / 180) + 1.0 / math.cos(lat*math.pi / 180)) / math.pi) / 2 * 2**zoom)

    return x, y, zoom




In [29]:
WGS84ToTile( 8.009742, 47.498308, 18)

(136904, 91668, 18)

In [30]:
x, y, z = WGS84ToTile( 8.009742, 47.498308, 18)


s = f"https://a.tile.openstreetmap.org/{z}/{x}/{y}.png"
print(s)

https://a.tile.openstreetmap.org/18/136904/91668.png
